In [1]:
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git" --q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:0

In [2]:
import torch
from unsloth import FastLanguageModel

# Check CUDA compatibility
major_version, minor_version = torch.cuda.get_device_capability()

# Instantiate FastLanguageModel
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="google/gemma-2b-it",
    max_seq_length=4096,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

/usr/local/lib/python3.10/dist-packages/unsloth/__init__.py:72: UserWarning: Unsloth: Running `ldconfig /usr/lib64-nvidia` to link CUDA.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Unsloth: You passed in `google/gemma-2b-it` and `load_in_4bit = True`.
We shall load `unsloth/gemma-2b-bnb-4bit` for 4x faster loading.


config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2024.5 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.


In [4]:
prompt = """Based on given instruction and context, generate an appropriate response

### Instruction:
{}

### Context:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    contexts = examples["context"]
    responses = examples["response"]
    texts = []

    for i,j,k  in zip(instructions, contexts,responses):
        text = prompt.format(i,j,k) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("databricks/databricks-dolly-15k", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True)

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 2,
        max_steps = 10,
        learning_rate = 0.0005,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

# Training
trainer_stats = trainer.train()

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/15011 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 15,011 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 10
 "-____-"     Number of trainable parameters = 19,611,648


Step,Training Loss
1,2.509200
2,2.560600
3,2.429800
4,2.200600
5,2.033900
6,1.747500
7,1.849700
8,1.741900
9,1.635300
10,1.733600


In [6]:
inputs = tokenizer(
    [
        prompt.format(
            "Provide a detailed explanation of the events and circumstances that led to the outbreak of World War II.",  # instruction
            " The goal is to offer a clear and informative account of the factors, political decisions, and international tensions that played a crucial role in triggering World War II. Ensure that the explanation covers the period leading up to the war, key events, and the involvement of major nations",  # context
            " ",  # response
        )
    ] * 1,
    return_tensors="pt",
).to("cuda")

# Generate response
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=1028)

# Output:
# The outbreak of World War II was triggered by a series of events that began
# in the early 1930s. The Great Depression of the 1930s led to a decline in global economic activity,
# which in turn led to a decline in trade and a rise in nationalist sentiment.
# This led to a series of diplomatic tensions between nations, particularly between Germany and Britain.
# Germany's aggressive foreign policy and expansionist ambitions led to a series of diplomatic
# incidents and military provocations, culminating in the invasion of Poland in September 1939.
# This led to the outbreak of World War II, with Germany and Britain declaring war
# on Poland and France declaring war on Germany.
# The war quickly escalated into a global conflict, with nations from across the world involved in the conflict.
# The war was characterized by massive military mobilization, mass destruction, and the use of new weapons such as atomic bombs and nuclear weapons.
# The war ended in 1945 with the defeat of Germany and Japan, and the establishment of the United Nations as a global organization to prevent future conflicts.

<bos>Based on given instruction and context, generate an appropriate response

### Instruction:
Provide a detailed explanation of the events and circumstances that led to the outbreak of World War II.

### Context:
 The goal is to offer a clear and informative account of the factors, political decisions, and international tensions that played a crucial role in triggering World War II. Ensure that the explanation covers the period leading up to the war, key events, and the involvement of major nations

### Response:
 
The outbreak of World War II was triggered by a series of events that began in the early 1930s. The Great Depression of the 1930s led to a decline in global economic activity, which in turn led to a decline in trade and a rise in nationalist sentiment. This led to a series of diplomatic tensions between nations, particularly between Germany and Britain. Germany's aggressive foreign policy and expansionist ambitions led to a series of diplomatic incidents and military provo